In [6]:
import json
from json_repair import repair_json

In [ ]:
sample_message = """{"message_id": 31329, "text": "expande tu imagen ! ! ! ! ! ! ! ! ! si no puede ver este mail , entre a : http : / / www . supermedios . com / admin / mailing / proyecto . php ? id = 160\neste mensaje se enva bajo los artculos 2 y 4 de la ley\n19 . 628 y 28 b de la ley 19 . 955 de la constitucin de la repblica\nde chile actualizada el 14 de julio 2004 . su direccin ha sido extrada\nmanualmente por personal de nuestra compaa desde su sitio\nweb en internet , o ha sido introducida por usted al aceptar el envo\nde mensajes publicitarios al inscribirse en alguno de los sitios o foros\nde nuestra red de trabajo .\npara ser removido presione borrar", "label": 1, "label_text": "spam", "subject": "expande tu imagen ! ! ! ! ! ! ! ! !", "message": "si no puede ver este mail , entre a : http : / / www . supermedios . com / admin / mailing / proyecto . php ? id = 160\neste mensaje se enva bajo los artculos 2 y 4 de la ley\n19 . 628 y 28 b de la ley 19 . 955 de la constitucin de la repblica\nde chile actualizada el 14 de julio 2004 . su direccin ha sido extrada\nmanualmente por personal de nuestra compaa desde su sitio\nweb en internet , o ha sido introducida por usted al aceptar el envo\nde mensajes publicitarios al inscribirse en alguno de los sitios o foros\nde nuestra red de trabajo .\npara ser removido presione borrar\n", "date": "2005-01-19"}"""
sample_messaage = repair_json(sample_message)
sample_message_json = json.loads(sample_messaage)
formatted_message_json = json.dumps(sample_message_json, indent=2)
print(formatted_message_json)

{
  "message_id": 31329,
  "text": "expande tu imagen ! ! ! ! ! ! ! ! ! si no puede ver este mail , entre a : http : / / www . supermedios . com / admin / mailing / proyecto . php ? id = 160\neste mensaje se enva bajo los artculos 2 y 4 de la ley\n19 . 628 y 28 b de la ley 19 . 955 de la constitucin de la repblica\nde chile actualizada el 14 de julio 2004 . su direccin ha sido extrada\nmanualmente por personal de nuestra compaa desde su sitio\nweb en internet , o ha sido introducida por usted al aceptar el envo\nde mensajes publicitarios al inscribirse en alguno de los sitios o foros\nde nuestra red de trabajo .\npara ser removido presione borrar",
  "label": 1,
  "label_text": "spam",
  "subject": "expande tu imagen ! ! ! ! ! ! ! ! !",
  "message": "si no puede ver este mail , entre a : http : / / www . supermedios . com / admin / mailing / proyecto . php ? id = 160\neste mensaje se enva bajo los artculos 2 y 4 de la ley\n19 . 628 y 28 b de la ley 19 . 955 de la constitucin de la repb

In [10]:
# Count elements in test.jsonl
test_list = []
with open('test.jsonl', 'r') as f:
    for line in f:
        test_item = json.loads(repair_json(line))
        test_list.append(test_item)


# Count elements in train.jsonl
train_list = []
with open('train.jsonl', 'r') as f:
    for line in f:
        train_item = json.loads(repair_json(line))
        train_list.append(train_item)

print(f"Number of elements in test.jsonl: {len(test_list)}")
print(f"Number of elements in train.jsonl: {len(train_list)}")

Number of elements in test.jsonl: 2000
Number of elements in train.jsonl: 31716


In [11]:
from collections import Counter

# Count label_text values in test set
test_label_counts = Counter()
for item in test_list:
    label = item.get('label_text', 'unknown')
    test_label_counts[label] += 1

# Count label_text values in train set
train_label_counts = Counter()
for item in train_list:
    label = item.get('label_text', 'unknown')
    train_label_counts[label] += 1

# Display statistics
print("=" * 60)
print("TEST SET STATISTICS")
print("=" * 60)
print(f"Total messages: {len(test_list)}")
for label, count in sorted(test_label_counts.items()):
    percentage = (count / len(test_list)) * 100
    print(f"  {label}: {count:,} ({percentage:.2f}%)")

print("\n" + "=" * 60)
print("TRAIN SET STATISTICS")
print("=" * 60)
print(f"Total messages: {len(train_list)}")
for label, count in sorted(train_label_counts.items()):
    percentage = (count / len(train_list)) * 100
    print(f"  {label}: {count:,} ({percentage:.2f}%)")

print("\n" + "=" * 60)
print("COMBINED STATISTICS")
print("=" * 60)
combined_total = len(test_list) + len(train_list)
print(f"Total messages: {combined_total:,}")

# Combine counts
combined_counts = Counter()
for label in set(list(test_label_counts.keys()) + list(train_label_counts.keys())):
    combined_counts[label] = test_label_counts[label] + train_label_counts[label]

for label, count in sorted(combined_counts.items()):
    percentage = (count / combined_total) * 100
    print(f"  {label}: {count:,} ({percentage:.2f}%)")

# Verify totals match
print("\n" + "=" * 60)
print("VERIFICATION")
print("=" * 60)
test_sum = sum(test_label_counts.values())
train_sum = sum(train_label_counts.values())
combined_sum = sum(combined_counts.values())

print(f"Test labels sum: {test_sum:,} (matches count: {test_sum == len(test_list)})")
print(f"Train labels sum: {train_sum:,} (matches count: {train_sum == len(train_list)})")
print(f"Combined sum: {combined_sum:,} (matches total: {combined_sum == combined_total})")


TEST SET STATISTICS
Total messages: 2000
  ham: 992 (49.60%)
  spam: 1,008 (50.40%)

TRAIN SET STATISTICS
Total messages: 31716
  ham: 15,553 (49.04%)
  spam: 16,163 (50.96%)

COMBINED STATISTICS
Total messages: 33,716
  ham: 16,545 (49.07%)
  spam: 17,171 (50.93%)

VERIFICATION
Test labels sum: 2,000 (matches count: True)
Train labels sum: 31,716 (matches count: True)
Combined sum: 33,716 (matches total: True)
